In [2]:
import sys
sys.path.append('/work/yunruili/2dNMR_project_GNN/')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from load_alignment_graph_nmr import graph_nmr_data, custom_collate_fn
from Comenet_NMR import ComENet
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.cuda.amp import autocast, GradScaler
import time
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os


/work/yunruili/anaconda_env/diffusion/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### load multi task model

In [3]:
sys.path.append('/work/yunruili/2dNMR_project_GNN/')
from load_graph_cnmr_hnmr_alignment import graph_nmr_alignment_data, custom_collate_fn, CustomBatchSampler
from Comenet_NMR_multitask import ComENet

graph_path = '/scratch0/haox/2DNMR_prediction_gt/Datasets/graph3d/'
nmr_path = '/scratch0/haox/yunruili/'
# cnmr_path = '/scratch0/haox/yunruili/cnmr_alignment'
# hnmr_path = '/scratch0/haox/yunruili/hnmr_alignment'
csv_cnmr = '../filtered_cnmr_smile_dataset_22k.csv'
csv_hnmr = '../filtered_hnmr_smile_dataset_67.csv'
csv_common = '../filtered_common_smile_dataset_1600.csv'
dataset_c = graph_nmr_alignment_data(csv_cnmr, graph_path, nmr_path, type='c')
dataset_h = graph_nmr_alignment_data(csv_hnmr, graph_path, nmr_path, type='h')
dataset_ch = graph_nmr_alignment_data(csv_common, graph_path, nmr_path, type='both')

# Set the seed for reproducibility
torch.manual_seed(0)  # Replace your_seed with your chosen seed value

# Define the proportions or absolute sizes for your train and val sets
train_size_c = int(0.8 * len(dataset_c))
val_size_c = len(dataset_c) - train_size_c

train_size_h = int(0.8 * len(dataset_h))
val_size_h = len(dataset_h) - train_size_h

train_size_ch = int(0.8 * len(dataset_ch))
val_size_ch = len(dataset_ch) - train_size_ch

# Split the datasets
train_dataset_c, val_dataset_c = random_split(dataset_c, [train_size_c, val_size_c])
train_dataset_h, val_dataset_h = random_split(dataset_h, [train_size_h, val_size_h])
train_dataset_ch, val_dataset_ch = random_split(dataset_ch, [train_size_ch, val_size_ch])

# Create DataLoaders for training and validation sets
train_dataloader_c = DataLoader(train_dataset_c, batch_size=1, shuffle=True, collate_fn=custom_collate_fn)
val_dataloader_c = DataLoader(val_dataset_c, batch_size=1, shuffle=True, collate_fn=custom_collate_fn)

train_dataloader_h = DataLoader(train_dataset_h, batch_size=1, shuffle=True, collate_fn=custom_collate_fn)
val_dataloader_h = DataLoader(val_dataset_h, batch_size=1, shuffle=True, collate_fn=custom_collate_fn)

train_dataloader_ch = DataLoader(train_dataset_ch, batch_size=1, shuffle=True, collate_fn=custom_collate_fn)
val_dataloader_ch = DataLoader(val_dataset_ch, batch_size=1, shuffle=True, collate_fn=custom_collate_fn)

# Now you can create custom loaders for each set
train_custom_loader = CustomBatchSampler(train_dataloader_c, train_dataloader_ch, train_dataloader_h, n1=7, n2=80)
val_custom_loader = CustomBatchSampler(val_dataloader_c, val_dataloader_ch, val_dataloader_h, n1=14, n2=141)

In [4]:
hc = 256
chc=[128]
hhc=[128, 64, 64]
n = 3
n_out = 2
model = ComENet(in_embed_size=3, out_channels=1, agg_method='sum',\
     hidden_channels=hc, c_out_hidden=chc, h_out_hidden=hhc, num_layers=n, num_output_layers=n_out)
msg = model.load_state_dict(torch.load(
    '../gnn3d_multi_align_sum_hiddendim_%d_nlayers_%d_noutlayers_%d_couthidden_%s_houthidden_%s.pt'\
    %(hc, n, n_out, ''.join(str(i) for i in chc), ''.join(str(i) for i in hhc)), map_location='cpu'))
print(msg)



<All keys matched successfully>


#### apply on 2dnmr data

In [5]:
# evaluate on 2d NMR data
from torch_geometric.data import DataLoader as loader_2dnmr
from torch.utils.data import Dataset
import pandas as pd

class graph_nmr_data_2d(Dataset):
    '''Returns the index of non-zero values on y-axis and the corresponding x-axis'''
    def __init__(self, csv_file, graph_path, nmr_path, x='C'):
        df = pd.read_csv(csv_file)
        self.file_list = df['File_name'].to_list()
        # filter out Test_*.csv
        self.file_list = [x for x in self.file_list if not x.startswith('Test_')]

        self.nmr_path = nmr_path
        self.graph_path = graph_path
        self.x = x
    def __len__(self):
        return len(self.file_list)
    def __getitem__(self, item):
        filename = self.file_list[item].split('.')[0]

        graph_file = os.path.join(self.graph_path, filename + '.pickle')
        graph_data = pickle.load(open(graph_file, 'rb'))
        graph_data.x = graph_data.x.float()
        
        graph_data.has_c = True
        graph_data.has_h = True

        # use original csv files so that the C nodes are not duplicated
        nmr_file = os.path.join(self.nmr_path, filename + '.csv')
        nmr_data = pd.read_csv(nmr_file)

        # Forward fill the 'No.' column to fill empty values with the previous row's value
        if 'No.' in nmr_data.columns:
            nmr_data['No.'].fillna(method='ffill', inplace=True)
            # Group by both 'No.' and '13C' columns and create a list of '1H' values
            grouped = nmr_data.groupby(['No.', '13C'])['1H'].apply(list).reset_index()
        else:
            grouped = nmr_data.groupby('13C')['1H'].apply(list).reset_index()
        c_list = grouped['13C'].tolist()
        h_list = grouped['1H'].apply(lambda x: x if len(x) > 1 else [x[0], x[0]]).tolist()
        c_peaks = torch.tensor(c_list).float().squeeze()/200
        try:
            h_peaks = torch.tensor(h_list).float()/10
        except Exception as e:
            print(f"Error while converting to tensor for filename: {filename}")
            print(e)
            h_peaks = torch.tensor([])


        # this deals with the 1d mapped out file
        # if self.x == 'C':
        #     nonzero_nmr = nmr_data[nmr_data['1H']!=0]
        # else:
        #     nonzero_nmr = nmr_data[nmr_data['13C']!=0]
        # c_peaks = torch.tensor(nonzero_nmr['13C'].tolist())
        # h_peaks = torch.tensor(nonzero_nmr['1H'].tolist())

        return graph_data, c_peaks, h_peaks, filename

In [6]:
graph_path_2dnmr = '/scratch0/yunruili/2dnmr_30k/graph_3d/'
csv_file_2dnmr = '../nmr_smile_solvent_filtered2_3dgnn.csv'
nmr_path_2dnmr = '/scratch0/yunruili/2dnmr_30k/csv_30k/'
save_dir = '/scratch0/yunruili/2dnmr_30k/nmr_2dcsv_chmatched'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

dataset_2dnmr = graph_nmr_data_2d(csv_file_2dnmr, graph_path_2dnmr, nmr_path_2dnmr)
dataloader_2dnmr = loader_2dnmr(dataset_2dnmr, shuffle=True, batch_size=1)

list_files = []
for i, data in enumerate(dataloader_2dnmr):
    graph, cnmr, hnmr, filename = data
    
    c_nodes = (graph.x[:,0]==5).nonzero(as_tuple=True)[0]
    h_nodes = (graph.x[:, 0] == 0).nonzero(as_tuple=True)[0] 

    c_shifts, h_shifts = model(graph)

    ##### calculate the indices of C node connected to H
    # Initialize a list to store C nodes connected to H
    c_nodes_connected_to_h = []
    # Check each C node for connection to any H node
    for c_node in c_nodes:
        # Get indices of edges involving the C node
        edges_of_c = (graph.edge_index[0] == c_node) | (graph.edge_index[1] == c_node)

        # Get all nodes that are connected to this C node
        connected_nodes = torch.cat((graph.edge_index[0][edges_of_c], graph.edge_index[1][edges_of_c])).unique()

        # Check if any of these connected nodes are H nodes
        if any(node in h_nodes for node in connected_nodes):
            c_nodes_connected_to_h.append(c_node.item())
    
    # Convert to a tensor
    c_nodes_connected_to_h = torch.tensor(c_nodes_connected_to_h)

    c_shifts = c_shifts.flatten().detach().numpy()
    h_shifts = h_shifts.flatten().detach().numpy()
    cnmr = cnmr.squeeze().detach().numpy()
    hnmr = hnmr.squeeze().detach().numpy()

    c_index = [i for i, x in enumerate(c_nodes) if x in c_nodes_connected_to_h]
    c_shifts = c_shifts[c_index]

    print(filename)
    print(len(c_shifts), len(h_shifts))
    print(len(cnmr), len(hnmr))

    # make c-h pairs for ground truth and prediction
    hnmr = np.mean(hnmr, axis=1)
    pred = np.stack([c_shifts, h_shifts], axis=1)
    gt = np.stack([cnmr, hnmr], axis=1)

    if i ==0:
        break




/work/yunruili/anaconda_env/diffusion/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


('33291',)
5 5
5 5


#### matching

In [10]:
## hungarian algorithm, for same length 
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist
from matching_code.ARG import ARG

def match_samelen(pred, gt):
    # Calculate the cost (distance) matrix
    cost_matrix = cdist(pred, gt, metric='euclidean')

    # Solve the assignment problem
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    # Output the matched pairs
    matched_pairs = np.array(list(zip(row_ind, col_ind)))
    return matched_pairs

In [89]:
from sklearn.preprocessing import normalize
## graduate assignment algorithm, for different length
def compatibility(atr1, atr2):
    #Consider the order to return 0 or inf
    
    if np.isnan(atr1).any() or np.isnan(atr1).any():
        return 0
    if (atr1 == float('Inf')).any() or (atr2 == float('Inf')).any():
        return float('Inf')
    if len(atr1) != len(atr2):
        return 0
    if (atr1 == 0).all() or (atr2 == 0).all():
        return 0
    
    
    dim = len(atr1)
    score = 1-((atr1-atr2)**2).sum()
    #score = atr1 * atr2
    return score

def pre_compute_compatibility(ARG1, ARG2, alpha=1, stochastic=0, node_binary=True, edge_binary=True, dist_mask=None):
    '''
    Compute the best matching with two ARGs.
    
    '''
    
    # Size of the real match-in matrix
    A = ARG1.num_nodes
    I = ARG2.num_nodes
    real_size = [A, I] # ???
    augment_size = [A+1, I+1] # size of the matrix with slacks
    
    #### compute c_aibj ####
    # nil node compatibility percentage
    prct = 10
    
    ## pre-calculate the node compatibility
    C_n = np.zeros(augment_size)
    
    if node_binary:
        C_n[:A,:I] = cdist(ARG1.nodes_vector, ARG2.nodes_vector, compatibility_binary)
    else:
        C_n[:A,:I] = cdist(ARG1.nodes_vector, ARG2.nodes_vector, compatibility)
    
    # Add score to slacks
    C_n[A,:-1] =  np.percentile(C_n[:A,:I],prct,0)
    C_n[:-1,I] =  np.percentile(C_n[:A,:I],prct,1)
    C_n[A,I] = 0 

    # times the alpha weight
    C_n = alpha*C_n
    
#     print(C_n)
    
    if dist_mask is not None:
        C_n[:A,:I] = np.multiply(C_n[:A,:I], dist_mask)
    
    return C_n

def graph_matching(C_n, ARG1, ARG2, beta_0=0.1, beta_f=20, beta_r=1.025, 
                   I_0=20, I_1=200, e_B=0.1, e_C=0.01, fixed_match = None):  # C_e, 
    ### fixed match is a matrix (A*I) for the pre-determined matching pairs 
    ##  We first do not consider the stochastic.
    # set up the soft assignment matrix
    
    A = C_n.shape[0] - 1
    I = C_n.shape[1] - 1 
    m_Head = np.random.rand(A+1, I+1) # Not an assignment matrix. (Normalized??)
    m_Head[-1,-1] = 0
    
    ### zero the nodes that already matched 
    if fixed_match is not None:
        print('fixed some points')
        C_n = np.multiply(C_n, fixed_match)

    # Initialization for parameters

    ## beta is the penalty parameters
    # includes beta_0, beta_f, beta_r

    ## I controls the maximum iteration for each round
    # includes I_0 and I_1

    ## e controls the range
    # includes e_B and e_C

    # begin matching
    beta = beta_0

    stochastic = False ### we first do not consider this case
    
    # the indexes of the non-zero elements in C_n
    idx1 = np.unique(C_n.nonzero()[0])
    idx2 = np.unique(C_n.nonzero()[1])  # not used, only check number
#     print(A, len(idx1))
#     print(I, len(idx2))
    
#     tmp_edges_1 = np.full([(A+1),(A+1)], np.nan)
#     tmp_edges_1[:A,:A] = ARG1.edges_matrix
#     tmp_edges_1[A,A] = float('Inf')
    
#     tmp_edges_2 = np.full([(I+1),(I+1)], np.nan)
#     tmp_edges_2[:I,:I] = ARG2.edges_matrix
#     tmp_edges_2[I,I] = float('Inf')

    while beta < beta_f:

        ## Round-B
        #check if converges
        converge_B = False
        I_B = 0
        while (not converge_B) and I_B <= I_0: # Do B until B is converge or iteration exceeds
            if stochastic:
                m_Head = m_Head ### + ???           

#             print('I_B', m_Head[0])
            old_B = m_Head # the old matrix
            I_B += 1 

            # Build the partial derivative matrix Q
            Q = np.zeros([A+1, I+1])
            
            #### only calculate Q for the indexes positions

            # Edge attribute
#             for a in idx1:
#                 ## get the non-zero (potential matches) in the second graph
#                 idx2 = np.unique(np.nonzero(C_n[a]))
#                 for i in idx2:
#                     c_e = np.zeros([A+1, I+1])
#                     edge1 = tmp_edges_1[a]
#                     edge2 = tmp_edges_2[i]
#                     c_e = np.dot(edge1, np.transpose(edge2))
#                     c_e[np.isnan(c_e)] = 0
#                     c_e[c_e == float('Inf')] = 0
#                     Q[a,i] = sum(sum(c_e*m_Head))

            # Node attribute
            Q = Q + C_n 
#             print(Q)

            # Update m_Head
            m_Head = np.exp(beta*Q) 
            m_Head[-1, -1] = 0
            
#             print(m_Head)
            
            converge_C = False
            I_C = 0
            while (not converge_C) and I_C <= I_1: # Do C until C is converge or iteration exceeds
                I_C += 1
                old_C = m_Head
                
#                 print(m_Head[0])

                # Begin alternative normalization. 
                # Do not consider the row or column of slacks
                # by column
                m_Head = normalize(m_Head, norm='l2',axis=0)*normalize(m_Head, norm='l2',axis=0)
                # By row
                m_Head = normalize(m_Head, norm='l2',axis=1)*normalize(m_Head, norm='l2',axis=1)
                
#                 print('I_C', m_Head[0])

                # print(sum(m_Head))
                # update converge_C
                converge_C = abs(sum(sum(m_Head-old_C))) < e_C

            # update converge_B
            converge_B = abs(sum(sum(m_Head[:A,:I]-old_B[:A,:I]))) < e_B
        # update beta
        beta *= beta_r

    match_matrix = heuristic(m_Head, A, I)
    #match_matrix = m_Head
    return match_matrix

def heuristic(M, A, I):
    '''
    Make a soft assignment matrix to a permutation matrix. 
    Due to some rules.
    We just set the maximum element in each column to 1 and 
    all others to 0.
    This heuristic will always return a permutation matrix 
    from a row dominant doubly stochastic matrix.
    '''
    M = normalize(M, norm='l2',axis=1)*normalize(M, norm='l2',axis=1)
    for i in range(A+1):
        index = np.argmax(M[i,:]) # Get the maximum index of each row
        M[i,:] = 0
#         if index != I-1:
#             M[:,index] = 0
#         ###
#         else:
#             print(i, I-1)
        M[i,index] = 1
    M = M[:A,:I]
    return M

def match_difflen(pred, gt):
    edges_pred = np.zeros((len(pred), len(pred)))
    edges_gt = np.zeros((len(gt), len(gt)))

    dist_matrix = cdist(pred, gt)
    ARG1 = ARG(edges_pred, pred)
    ARG2 = ARG(edges_gt, gt)
    start_time = time.time()
    C_n = pre_compute_compatibility( ARG1, ARG2, alpha=1, stochastic=0,node_binary=False)
    # print("--- Calculate C_n,  %s hours ---" % ((time.time() - start_time)/3600))
    start_time = time.time()
    match_matrix = graph_matching(C_n=C_n, ARG1 = ARG1, ARG2 = ARG2, 
                                beta_0=0.1, beta_f=100, beta_r=1.01, 
                                I_0=200, I_1=200, e_B=0.00005, e_C=0.00005
                                )


    g1, g2 = match_matrix.nonzero()
    rslt = np.full([match_matrix.shape[0], 2], -1) # need to maintain record if one node is graph 1 is not matched

    rslt[:, 0] = np.arange(0, match_matrix.shape[0])


    for i in range(len(g1)):
        rslt[g1[i], 1] = g2[i] 
    
    # Find rows in array1 where the second column is -1
    rows_to_match = np.where(rslt[:, 1] == -1)[0]
    # For each of these rows, find the index of the minimum value in array2 and assign it
    for row in rows_to_match:
        rslt[row, 1] = np.argmin(dist_matrix[row])
    return rslt



In [23]:
match_rslt(pred,gt)

--- Calculate C_n,  9.773837195502386e-07 hours ---


array([[ 0,  4],
       [ 1, -1],
       [ 2,  2],
       [ 3,  0],
       [ 4,  3]])

In [25]:
match_rslt(pred,gt)

--- Calculate C_n,  1.6628371344672308e-06 hours ---


array([[0, 4],
       [1, 1],
       [2, 2],
       [3, 0],
       [4, 3]])

In [28]:
match_samelen(pred, gt)

array([[0, 4],
       [1, 1],
       [2, 2],
       [3, 0],
       [4, 3]])

In [26]:
pred

array([[0.2765539 , 0.39952013],
       [0.5683851 , 0.6435527 ],
       [0.5973886 , 0.74988073],
       [0.56094277, 0.74365056],
       [0.38517028, 0.51225513]], dtype=float32)

In [27]:
gt

array([[0.5553    , 0.6967    ],
       [0.57525   , 0.67340004],
       [0.5968    , 0.6802    ],
       [0.36099997, 0.49019998],
       [0.2783    , 0.37449998]], dtype=float32)

In [90]:
graph_path_2dnmr = '/scratch0/yunruili/2dnmr_30k/graph_3d/'
csv_file_2dnmr = '../nmr_smile_solvent_filtered2_3dgnn.csv'
nmr_path_2dnmr = '/scratch0/yunruili/2dnmr_30k/csv_30k/'
save_dir = '/scratch0/yunruili/2dnmr_30k/nmr_2dcsv_chmatched'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

dataset_2dnmr = graph_nmr_data_2d(csv_file_2dnmr, graph_path_2dnmr, nmr_path_2dnmr)
dataloader_2dnmr = loader_2dnmr(dataset_2dnmr, shuffle=True, batch_size=1)

list_files = []
for i, data in enumerate(dataloader_2dnmr):
    graph, cnmr, hnmr, filename = data
    
    c_nodes = (graph.x[:,0]==5).nonzero(as_tuple=True)[0]
    h_nodes = (graph.x[:, 0] == 0).nonzero(as_tuple=True)[0] 

    c_shifts, h_shifts = model(graph)

    ##### calculate the indices of C node connected to H
    # Initialize a list to store C nodes connected to H
    c_nodes_connected_to_h = []
    # Check each C node for connection to any H node
    for c_node in c_nodes:
        # Get indices of edges involving the C node
        edges_of_c = (graph.edge_index[0] == c_node) | (graph.edge_index[1] == c_node)

        # Get all nodes that are connected to this C node
        connected_nodes = torch.cat((graph.edge_index[0][edges_of_c], graph.edge_index[1][edges_of_c])).unique()

        # Check if any of these connected nodes are H nodes
        if any(node in h_nodes for node in connected_nodes):
            c_nodes_connected_to_h.append(c_node.item())
    
    # Convert to a tensor
    c_nodes_connected_to_h = torch.tensor(c_nodes_connected_to_h)

    c_shifts = c_shifts.flatten().detach().numpy()
    h_shifts = h_shifts.flatten().detach().numpy()
    cnmr = cnmr.squeeze().detach().numpy()
    hnmr = hnmr.squeeze().detach().numpy()

    c_index = [i for i, x in enumerate(c_nodes) if x in c_nodes_connected_to_h]
    c_shifts = c_shifts[c_index]

    # print(filename)
    # print(len(c_shifts), len(h_shifts))
    # print(len(cnmr), len(hnmr))

    # make c-h pairs for ground truth and prediction
    hnmr_mean = np.mean(hnmr, axis=1)
    pred = np.stack([c_shifts, h_shifts], axis=1)
    gt = np.stack([cnmr, hnmr_mean], axis=1)

    try:
        if len(cnmr) == len(c_shifts):
            rslt = match_samelen(pred, gt)
        else:
            rslt = match_difflen(pred, gt)

        cnmr_expanded = np.array([[cnmr[i]] for i in rslt[:,1]])
        hnmr_expanded = np.array([hnmr[i] for i in rslt[:, 1]])

        # save as pickle
        c_name = os.path.join(save_dir, '%s_c.pickle'%filename)
        h_name = os.path.join(save_dir, '%s_h.pickle'%filename)
        
        # Writing to a pickle file
        with open(c_name, 'wb') as file:
            pickle.dump(cnmr_expanded, file)
        with open(h_name, 'wb') as file:
            pickle.dump(hnmr_expanded, file)
    except Exception as e:
        print(filename)
        list_files.append(filename)
        print(e)

/work/yunruili/anaconda_env/diffusion/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [85]:
len(dataloader_2dnmr)

24416

In [86]:
hnmr_expanded

array([[0.21300001, 0.21300001],
       [0.21300001, 0.21300001],
       [0.21300001, 0.21300001],
       [0.10399999, 0.10399999],
       [0.43      , 0.361     ],
       [0.158     , 0.158     ],
       [0.28399998, 0.26      ],
       [0.191     , 0.191     ],
       [0.21300001, 0.21300001],
       [0.102     , 0.102     ],
       [0.43      , 0.361     ],
       [0.21300001, 0.21300001],
       [0.372     , 0.372     ],
       [0.41399997, 0.35      ],
       [0.187     , 0.187     ],
       [0.10399999, 0.10399999],
       [0.21300001, 0.21300001],
       [0.28399998, 0.26      ],
       [0.148     , 0.148     ],
       [0.41399997, 0.35      ],
       [0.158     , 0.158     ],
       [0.102     , 0.102     ],
       [0.21300001, 0.21300001],
       [0.28599998, 0.272     ],
       [0.21300001, 0.21300001]], dtype=float32)

In [76]:
c_shifts

array([0.16011375, 0.20083353, 0.17291945, 0.06110011, 0.38393286,
       0.13929494, 0.06594548, 0.23464946, 0.17527401, 0.06670186,
       0.36907583, 0.16188505, 0.06013245, 0.36011192, 0.1695453 ,
       0.06381711, 0.2288571 , 0.11205356, 0.09719963, 0.36071712,
       0.1470994 , 0.04043383, 0.22088648, 0.06940529, 0.13266073],
      dtype=float32)

In [77]:
pred

array([[0.16011375, 0.22015314],
       [0.20083353, 0.31865034],
       [0.17291945, 0.26672673],
       [0.06110011, 0.12806502],
       [0.38393286, 0.40581673],
       [0.13929494, 0.19710767],
       [0.06594548, 0.24507643],
       [0.23464946, 0.3259301 ],
       [0.17527401, 0.23553558],
       [0.06670186, 0.10809404],
       [0.36907583, 0.40885496],
       [0.16188505, 0.26829177],
       [0.06013245, 0.4004858 ],
       [0.36011192, 0.36829948],
       [0.1695453 , 0.18305662],
       [0.06381711, 0.13314608],
       [0.2288571 , 0.30056593],
       [0.11205356, 0.22901818],
       [0.09719963, 0.13506696],
       [0.36071712, 0.39051753],
       [0.1470994 , 0.17446266],
       [0.04043383, 0.098152  ],
       [0.22088648, 0.3215965 ],
       [0.06940529, 0.26445144],
       [0.13266073, 0.20984401]], dtype=float32)

In [78]:
gt

array([[0.37      , 0.38199997],
       [0.176     , 0.21300001],
       [0.2375    , 0.187     ],
       [0.096     , 0.27899998],
       [0.1145    , 0.148     ],
       [0.0795    , 0.102     ],
       [0.086     , 0.372     ],
       [0.3725    , 0.3955    ],
       [0.1765    , 0.21300001],
       [0.2375    , 0.191     ],
       [0.096     , 0.27199998],
       [0.11149999, 0.158     ],
       [0.079     , 0.10399999]], dtype=float32)

In [79]:
rslt

array([[ 0,  8],
       [ 1,  8],
       [ 2,  1],
       [ 3, 12],
       [ 4,  7],
       [ 5, 11],
       [ 6, 10],
       [ 7,  9],
       [ 8,  8],
       [ 9,  5],
       [10,  7],
       [11,  1],
       [12,  6],
       [13,  0],
       [14,  2],
       [15, 12],
       [16,  8],
       [17, 10],
       [18,  4],
       [19,  0],
       [20, 11],
       [21,  5],
       [22,  8],
       [23,  3],
       [24,  1]])

In [53]:
rslt = match_difflen(pred, gt)
rslt

--- Calculate C_n,  2.545946174197727e-05 hours ---


array([[ 0, 12],
       [ 1, 24],
       [ 2, 16],
       [ 3, 12],
       [ 4, 12],
       [ 5, 11],
       [ 6,  4],
       [ 7, 17],
       [ 8,  5],
       [ 9, 18],
       [10, 23],
       [11, 18],
       [12,  6],
       [13, 18],
       [14,  1],
       [15, 22],
       [16,  9],
       [17, 24],
       [18, 16],
       [19, 14],
       [20, 20],
       [21,  8],
       [22,  2],
       [23, 10],
       [24, 19]])

In [55]:
rslt = match_difflen(pred, gt)
rslt

--- Calculate C_n,  2.4068421787685817e-05 hours ---


array([[ 0, 12],
       [ 1, 24],
       [ 2, 16],
       [ 3, 12],
       [ 4, 12],
       [ 5, 11],
       [ 6,  4],
       [ 7, 17],
       [ 8,  5],
       [ 9, 18],
       [10, 23],
       [11, 18],
       [12,  6],
       [13, 18],
       [14,  1],
       [15, 22],
       [16,  9],
       [17, 24],
       [18, 16],
       [19, 14],
       [20, 15],
       [21,  8],
       [22,  2],
       [23, 10],
       [24, 19]])